<a id="seccion-1"></a>

<h1 style="text-align: center; font-size: 70px; font-weight: 500;">Análisis de Indicadores: Sistema ETC Automatizado</h1>

- Autor: José Guillermo Sepúlveda Salazar.
- Practicante, Unidad de Estadística, UFRO, 2024.

<a id="seccion-2"></a>

## **Descripción**
Este documento detalla un enfoque metodológico sistemático destinado a la automatización de indicadores, que se extiende desde la adquisición de datos en línea hasta su cálculo y almacenamiento subsiguiente en Sistemas de Gestión de Bases de Datos (SGBD). Se ha implementado un marco metodológico que incluye una estructura de validación robusta para la extracción de archivos, garantizando su correcta integración en el repositorio designado. Además, realiza una verificación dentro del SGBD para asegurar la actualización de los indicadores correspondientes al año 2021.

Con el objetivo de reforzar la seguridad y asegurar la confidencialidad de los datos, el sistema incorpora mecanismos de autenticación avanzados. Esta estrategia no solo optimiza la eficiencia en la gestión de indicadores, sino que también protege la integridad de la información confidencial, cumpliendo así con los estándares de seguridad necesarios para el manejo de datos sensibles.

El proceso de automatización se estructura en tres fases fundamentales: extracción, transformación y carga (ETC). Durante la fase de extracción, el sistema verifica la presencia de la base de datos en el repositorio. En caso de no encontrarse, se procede a realizar una extracción de datos mediante técnicas de Web Scraping o conexiones API, lo que incluye la descarga y asignación de nombres a los archivos correspondientes. En la etapa de transformación, se llevan a cabo análisis estadísticos sobre cada indicador susceptible de actualización, lo que permite obtener resultados renovados. Finalmente, en la etapa de carga, el sistema se conecta al SGBD mediante credenciales de acceso específicas y verifica la actualización de los indicadores mediante la búsqueda del año correspondiente. Si los indicadores no están actualizados, se procede a la carga de los datos previamente transformados.

<a id="indice"></a>

## **Índice**
- [Portada](#seccion-1)
- [Descripción](#seccion-2)
- [Indicadores no aplicables](#seccion-3)
- [I) Extracción](#seccion-4)
    - [Obtenemos la BD de estudiantes matriculados en educación superior historica 2007-2023 de miFuturo.](#subseccion-4-1)
    - [Obtenemos la BD del personal académico del sistema de educación superior año 2022 de miFuturo.](#subseccion-4-2)
    
- [II) Transformación](#seccion-5)
    - [2. Evolución de matrículas anuales en pregrado y postgrado en universidades en chile.](#subseccion-5-2)
    - [5. Porcentaje de personas matriculadas en educación superior, según sexo respecto al total de personas matriculadas.](#subseccion-5-5)
    - [6. Distribución de matrículas de pregrado, magíster y doctorado según sexo.](#subseccion-5-6)
    - [7. Distribución de personas matriculadas en pregrado según sexo por campo de educación.](#subseccion-5-7)
    - [8. Distribución del personal académico de las instituciones de educación superior según sexo.](#subseccion-5-8)

- [III) Carga](#seccion-6)
    - [2. Evolución de matrículas anuales en pregrado y postgrado en universidades en chile.](#subseccion-6-2)
    - [5. Porcentaje de personas matriculadas en educación superior, según sexo respecto al total de personas matriculadas.](#subseccion-6-5)
    - [6. Distribución de matrículas de pregrado, magíster y doctorado según sexo.](#subseccion-6-6)
    - [7. Distribución de personas matriculadas en pregrado según sexo por campo de educación.](#subseccion-6-7)
    - [8. Distribución del personal académico de las instituciones de educación superior según sexo.](#subseccion-6-8)

<a id="seccion-3"></a>

## **Indicadores no aplicables**

- 1. **[Cantidad de programas de magíster y doctorado ofrecidos por universidades según región de sede (excluyendo la Región Metropolitana)](https://www.observa.minciencia.gob.cl/indicadores/formacion-y-capital-humano/numero-de-programas-de-magister-y-doctorado-por-region)**

- 3. **[Evolución anual de la cantidad de programas de postgrado ofrecidos por universidades en Chile, según región de sede.](https://www.observa.minciencia.gob.cl/indicadores/formacion-y-capital-humano/evolucion-programas-de-postgrado-por-region)**

- 4. **[Cantidad de programas de postgrado en áreas STEM por región de sede.](https://www.observa.minciencia.gob.cl/indicadores/formacion-y-capital-humano/porcentaje_matricula_regional_stem)**

[Volver al índice](#indice) 

<a id="seccion-4"></a>

# **I) Extracción**

[Volver al índice](#indice)

In [1]:
# Importamos las librerias necesarias
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import openpyxl
import time
import os
import pandas as pd
import numpy as np
import zipfile

<a id="subseccion-4-1"></a>

## [Obtenemos la BD de estudiantes matriculados en educación superior historica 2007-2023 de miFuturo.](https://www.mifuturo.cl/bases-de-datos-de-matriculados/)

[Volver al índice](#indice)

In [2]:
def esperar_descarga_completa(directorio, extension, timeout=300):
    tiempo_inicio = time.time()
    while True:
        archivos_temporales = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.crdownload') or archivo.endswith('.part')]
        if not archivos_temporales:
            # Espera un momento después de que desaparezcan los archivos temporales para asegurar que la descarga ha finalizado
            time.sleep(1)
            archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith(extension)]
            if archivos:
                return os.path.join(directorio, archivos[-1])  # Retorna el último archivo descargado con la extensión deseada
        elif (time.time() - tiempo_inicio) > timeout:
            raise Exception("Tiempo de espera para la finalización de la descarga excedido.")
        time.sleep(1)

# Función para obtener el último archivo descargado
def obtener_ultimo_archivo_descargado(directorio):
    lista_de_archivos = os.listdir(directorio)
    rutas_completas = [os.path.join(directorio, archivo) for archivo in lista_de_archivos]
    archivo_mas_reciente = max(rutas_completas, key=os.path.getmtime)
    return archivo_mas_reciente

nombre_del_archivo = "MATRICULA_SIES_ 2007_AL_2023.csv"     # Nombre del archivo a buscar
carpeta = os.getcwd()                                       # Utiliza el directorio actual de trabajo como carpeta de búsqueda
ruta_completa = os.path.join(carpeta, nombre_del_archivo)   # Construcción de la ruta del archivo

# Verificar si el archivo existe en la carpeta
if os.path.isfile(ruta_completa):
    print(f"El archivo '{nombre_del_archivo}' está presente en el directorio actual.")
else:
    print(f"El archivo '{nombre_del_archivo}' no se encontró en el directorio actual.")
    print("Procedemos al Scraping para obtener el descargable.")
    # Configurar opciones de Selenium y Chrome
    options = webdriver.ChromeOptions()

    # Especificar la carpeta de descarga como el directorio actual de trabajo
    current_directory = os.getcwd()
    prefs = {"download.default_directory": current_directory}
    options.add_experimental_option("prefs", prefs)

    # Inicializar el WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Acceder a la página web
    url = "https://www.mifuturo.cl/bases-de-datos-de-matriculados/"
    driver.get(url)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.TAG_NAME, "body")))       # Carga la página
    time.sleep(1)
    Click = WebDriverWait(driver, 0).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div > div > div:nth-child(7) > div.col-md-8.text-justify.col-content > div:nth-child(4) > div > p:nth-child(3) > a"))).click()
    time.sleep(1)
    # Esperar a que la descarga se complete
    ruta_archivo_descargado = esperar_descarga_completa(carpeta, '.zip')
    driver.quit()                                                                               # Cierra el navegador

    ultimo_archivo_descargado = obtener_ultimo_archivo_descargado(current_directory)            # Obtener el último archivo descargado
    print("Zip Descargado")
    # Extraer el contenido del archivo zip y renombrar el CSV
    with zipfile.ZipFile(ultimo_archivo_descargado, 'r') as archivo_zip:
        for miembro_zip in archivo_zip.namelist():
            if miembro_zip.endswith('.csv'):
                archivo_zip.extract(miembro_zip, carpeta)
                print("archivo descomprimido")
                # Renombrar el archivo CSV
                os.rename(os.path.join(carpeta, miembro_zip), os.path.join(carpeta, nombre_del_archivo))
                print(f"El archivo '{miembro_zip}' ha sido renombrado a '{nombre_del_archivo}'.")
                break 

    # Eliminar el archivo zip después de la extracción
    os.remove(ultimo_archivo_descargado)    
    print("carpeta zip eliminada")
   
    print("Descarga, extracción y renombrado completado")

El archivo 'MATRICULA_SIES_ 2007_AL_2023.csv' no se encontró en el directorio actual.
Procedemos al Scraping para obtener el descargable.
Zip Descargado
archivo descomprimido
El archivo 'MATRICULA OFICIAL 2007 AL 2023 09_06_2023 WEB.csv' ha sido renombrado a 'MATRICULA_SIES_ 2007_AL_2023.csv'.
carpeta zip eliminada
Descarga, extracción y renombrado completado


<a id="subseccion-4-2"></a>

## [Obtenemos la BD del personal académico del sistema de educación superior año 2022 de miFuturo.](https://www.mifuturo.cl/bases-de-datos-personal-academico/)

[Volver al índice](#indice)

In [3]:
def esperar_descarga_completa(directorio, extension, timeout=300):
    tiempo_inicio = time.time()
    while True:
        archivos_temporales = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.crdownload') or archivo.endswith('.part')]
        if not archivos_temporales:
            # Espera un momento después de que desaparezcan los archivos temporales para asegurar que la descarga ha finalizado
            time.sleep(1)
            archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith(extension)]
            if archivos:
                return os.path.join(directorio, archivos[-1])  # Retorna el último archivo descargado con la extensión deseada
        elif (time.time() - tiempo_inicio) > timeout:
            raise Exception("Tiempo de espera para la finalización de la descarga excedido.")
        time.sleep(1)

# Función para obtener el último archivo descargado
def obtener_ultimo_archivo_descargado(directorio):
    lista_de_archivos = os.listdir(directorio)
    rutas_completas = [os.path.join(directorio, archivo) for archivo in lista_de_archivos]
    archivo_mas_reciente = max(rutas_completas, key=os.path.getmtime)
    return archivo_mas_reciente

nombre_del_archivo = "Personal_Academico_SIES_2022.xlsx"    # Nombre del archivo a buscar
carpeta = os.getcwd()                                       # Utiliza el directorio actual de trabajo como carpeta de búsqueda
ruta_completa = os.path.join(carpeta, nombre_del_archivo)   # Construcción de la ruta del archivo

# Verificar si el archivo existe en la carpeta
if os.path.isfile(ruta_completa):
    print(f"El archivo '{nombre_del_archivo}' está presente en el directorio actual.")
else:
    print(f"El archivo '{nombre_del_archivo}' no se encontró en el directorio actual.")
    print("Procedemos al Scraping para obtener el descargable.")
    # Configurar opciones de Selenium y Chrome
    options = webdriver.ChromeOptions()

    # Especificar la carpeta de descarga como el directorio actual de trabajo
    current_directory = os.getcwd()
    prefs = {"download.default_directory": current_directory}
    options.add_experimental_option("prefs", prefs)

    # Inicializar el WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Acceder a la página web
    url = "https://www.mifuturo.cl/bases-de-datos-personal-academico/"
    driver.get(url)

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.TAG_NAME, "body")))       # Carga la página
    time.sleep(1)
    Click = WebDriverWait(driver, 0).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div > div > div:nth-child(7) > div.col-md-8.text-justify.col-content > div:nth-child(4) > div > p:nth-child(3) > a"))).click()
    time.sleep(1)
    # Esperar a que la descarga se complete
    ruta_archivo_descargado = esperar_descarga_completa(carpeta, '.xlsx')
    driver.quit()                                                                               # Cierra el navegador
    ultimo_archivo_descargado = obtener_ultimo_archivo_descargado(current_directory)            # Obtener el último archivo descargado
    os.rename(ultimo_archivo_descargado, os.path.join(current_directory, nombre_del_archivo))   # Renombrar el último archivo descargado
    
    print("Descarga y renombrado completo.")

El archivo 'Personal_Academico_SIES_2022.xlsx' no se encontró en el directorio actual.
Procedemos al Scraping para obtener el descargable.
Descarga y renombrado completo.


<a id="seccion-5"></a>

# **II) Transformación**

[Volver al índice](#indice) 

<a id="subseccion-5-2"></a>

## 2. **[Evolución de matrículas anuales en pregrado y postgrado en universidades en chile.](https://www.observa.minciencia.gob.cl/indicadores/formacion-y-capital-humano/evolucion-matricula-pregrado-magister-y-doctorado-en-universidades-en-chile)**

 * **Archivo**: MATRICULA_SIES_ 2007_AL_2023.csv
 * **Descripción**: En esta sección, se realiza la carga de un archivo CSV para convertirlo en un DataFrame. Posteriormente, se aplica un filtro para el año 2022. Mediante el uso de NumPy, se añade una columna adicional llamada "AUX_NIVEL" al DataFrame. Esta nueva columna se categoriza en base a los valores de "CARRERA CLASIFICACIÓN NIVEL 2". Finalmente, se exhiben los resultados obtenidos a través de la consola. Cabe destacar que la columna correspondiente al año se excluye en esta fase ya que en la carga de los datos se utiliza ese argumento.

[Volver al índice](#indice)    

In [4]:
# Leer el archivo CSV en un DataFrame de pandas
df_matricula = pd.read_csv("MATRICULA_SIES_ 2007_AL_2023.csv", encoding='ISO-8859-1', sep=';', low_memory=False) 

# Filtra los datos para el año 2022
df_filtrado = df_matricula[(df_matricula['AÑO'] == 'MAT_2022')]

# Define las columnas relevantes según los parámetros proporcionados sin el parámetro año
columnas_relevantes = [
    'TOTAL MATRICULADOS', 'MATRICULADOS MUJERES POR PROGRAMA',
    'MATRICULADOS HOMBRES POR PROGRAMA', 'CLASIFICACIÓN INSTITUCIÓN NIVEL 1',
    'CLASIFICACIÓN INSTITUCIÓN NIVEL 2', 'CLASIFICACIÓN INSTITUCIÓN NIVEL 3',
    'REGIÓN', 'NOMBRE CARRERA', 'ÁREA DEL CONOCIMIENTO', 'CINE-F 1997 ÁREA',
    'CINE-F 1997 SUBAREA', 'ÁREA CARRERA GENÉRICA', 'CINE-F 2013 ÁREA',
    'CINE-F 2013 SUBAREA', 'NIVEL GLOBAL', 'CARRERA CLASIFICACIÓN NIVEL 1',
    'CARRERA CLASIFICACIÓN NIVEL 2'
]

# Asegurarse de que df_matricula es una copia independiente
df_matricula = df_filtrado[columnas_relevantes].copy()

# Definir las condiciones
condiciones = [
    df_matricula['CARRERA CLASIFICACIÓN NIVEL 2'].isin(['Carreras Profesionales', 'Carreras Técnicas']),
    df_matricula['CARRERA CLASIFICACIÓN NIVEL 2'] == 'Magister',
    df_matricula['CARRERA CLASIFICACIÓN NIVEL 2'] == 'Doctorado',
    df_matricula['CARRERA CLASIFICACIÓN NIVEL 2'] == 'Postítulo'
]

# Definir las elecciones correspondientes a cada condición
elecciones = [
    'Pregrado',
    'Magister',
    'Doctorado',
    'No aplica'
]

# Crear la nueva columna 'AUX_NIVEL' con np.select
df_matricula['AUX_NIVEL'] = np.select(condiciones, elecciones, default='Otro')

# Mostrar el DataFrame modificado
print(df_matricula)

       TOTAL MATRICULADOS  MATRICULADOS MUJERES POR PROGRAMA  \
16051                  39                               17.0   
16052                 169                              114.0   
16053                  79                               63.0   
16054                 313                              252.0   
16055                   1                                1.0   
...                   ...                                ...   
31677                  25                               19.0   
31678                  39                               26.0   
31679                   8                                2.0   
31680                  12                                3.0   
31681                  13                                6.0   

       MATRICULADOS HOMBRES POR PROGRAMA CLASIFICACIÓN INSTITUCIÓN NIVEL 1  \
16051                               22.0                     Universidades   
16052                               55.0                     Universidades 

- ### **Grafica del indicador 2 :** Evolución de matrículas anuales en pregrado y postgrado en universidades en chile.

In [5]:
def calculo_distribucion(df):
    # Gráfico Evolución de matrículas anuales en pregrado y postgrado en universidades en chile
    matricula_pregrado =     df[(df['CLASIFICACIÓN INSTITUCIÓN NIVEL 1'] == 'Universidades') & (df['NIVEL GLOBAL'] == 'Pregrado')]['TOTAL MATRICULADOS'].sum()
    matricula_postgrado =     df[(df['CLASIFICACIÓN INSTITUCIÓN NIVEL 1'] == 'Universidades') & (df['NIVEL GLOBAL'] == 'Postgrado')]['TOTAL MATRICULADOS'].sum()

    # Cantidad de matrículas anuales de magíster y doctorado en la universidades en Chile.
    matricula_Dr =      df[(df['CLASIFICACIÓN INSTITUCIÓN NIVEL 1'] == 'Universidades') & (df['AUX_NIVEL'] == 'Doctorado')]['TOTAL MATRICULADOS'].sum()
    matricula_Mg =      df[(df['CLASIFICACIÓN INSTITUCIÓN NIVEL 1'] == 'Universidades') & (df['AUX_NIVEL'] == 'Magister')]['TOTAL MATRICULADOS'].sum()

    return matricula_pregrado, matricula_postgrado, matricula_Dr, matricula_Mg

# Llamar a la función y almacenar 
matricula_pregrado, matricula_postgrado, matricula_Dr, matricula_Mg = calculo_distribucion(df_matricula)

# Mostrar los resultados
print('Matriculas en pregrado en universidades en chile:    {:.0f} Personas'.format(matricula_pregrado))
print('Matriculas en postgrado en universidades en chile:   {:.0f} Personas\n'.format(matricula_postgrado))
print('Matriculas en doctorado en universidades en chile:   {:.0f} Personas'.format(matricula_Dr))
print('Matriculas en magístes en universidades en chile:    {:.0f} Personas'.format(matricula_Mg))

Matriculas en pregrado en universidades en chile:    683217 Personas
Matriculas en postgrado en universidades en chile:   54579 Personas

Matriculas en doctorado en universidades en chile:   6875 Personas
Matriculas en magístes en universidades en chile:    47704 Personas


In [6]:
# Obtener los primeros 5 resultados de df_matricula
primeros_cinco_resultados_matricula = df_matricula.head()

# Mostrar los primeros 5 resultados
print(primeros_cinco_resultados_matricula)

       TOTAL MATRICULADOS  MATRICULADOS MUJERES POR PROGRAMA  \
16051                  39                               17.0   
16052                 169                              114.0   
16053                  79                               63.0   
16054                 313                              252.0   
16055                   1                                1.0   

       MATRICULADOS HOMBRES POR PROGRAMA CLASIFICACIÓN INSTITUCIÓN NIVEL 1  \
16051                               22.0                     Universidades   
16052                               55.0                     Universidades   
16053                               16.0                     Universidades   
16054                               61.0                     Universidades   
16055                                NaN                     Universidades   

      CLASIFICACIÓN INSTITUCIÓN NIVEL 2 CLASIFICACIÓN INSTITUCIÓN NIVEL 3  \
16051            Universidades Privadas            Universidades Priv

<a id="subseccion-5-5"></a>

## 5. **[Porcentaje de personas matriculadas en educación superior, según sexo respecto al total de personas matriculadas.](https://www.observa.minciencia.gob.cl/genero/formacion-y-capital-humano/porcentaje-de-mujeres-matriculadas-en-la-educacion-superior-respecto-al-total)**

<a id="subseccion-5-6"></a>

## 6. **[Distribución de matrículas de pregrado, magíster y doctorado según sexo.](https://www.observa.minciencia.gob.cl/genero/formacion-y-capital-humano/porcentaje-de-mujeres-matriculadas-en-pregrado-magister-y-doctorado)**

<a id="subseccion-5-7"></a>

## 7. **[Distribución de personas matriculadas en pregrado según sexo por campo de educación.](https://www.observa.minciencia.gob.cl/genero/formacion-y-capital-humano/porcentaje-de-mujeres-matriculadas-en-pregrado-magister-y-doctorado-por-area-de-conocimiento)**

* **Archivo**: MATRICULA_SIES_ 2007_AL_2023.csv
* **Descripción**: En esta sección, se realiza la carga de un archivo CSV para convertirlo en un DataFrame. Posteriormente, se aplica un filtro para el año 2022. Mediante el uso de NumPy, se añade una columna adicional llamada "AUX_NIVEL" al DataFrame. Esta nueva columna se categoriza en base a los valores de "CARRERA CLASIFICACIÓN NIVEL 2". Para lograr una representación detallada de los datos de matriculación, el script duplica cada fila del DataFrame resultante para separar los datos de hombres y mujeres, asignando a cada uno la cantidad correspondiente de matriculados y ajustando la categorización de sexo. Las columnas originales de matriculados por género se eliminan después de este paso para evitar redundancias. El script finaliza combinando los DataFrames de hombres y mujeres en uno solo y eliminando cualquier fila que contenga valores nulos en la columna "Matriculados", mostrando los datos procesados por consola.  Cabe destacar que la columna correspondiente al año se excluye en esta fase ya que en la carga de los datos se utiliza ese argumento.

[Volver al índice](#indice) 

In [7]:
# Leer el archivo CSV en un DataFrame de pandas
df_matricula = pd.read_csv("MATRICULA_SIES_ 2007_AL_2023.csv", encoding='ISO-8859-1', sep=';', low_memory=False) 

# Filtra los datos para el año 2022 , que sean acreditadas excluyendo los postítulos
df_filtrado = df_matricula[(df_matricula['AÑO'] == 'MAT_2022') & 
                           (df_matricula['NIVEL GLOBAL'].isin(['Pregrado', 'Postgrado'])) &
                           (df_matricula['ACREDITACIÓN INSTITUCIONAL'] == 'ACREDITADA')]

# Define las columnas relevantes según los parámetros proporcionados sin el parámetro año
columnas_relevantes = [
    'MATRICULADOS MUJERES POR PROGRAMA', 'MATRICULADOS HOMBRES POR PROGRAMA',
    'CINE-F 2013 ÁREA', 'CARRERA CLASIFICACIÓN NIVEL 2'
]

# Asegurarse de que df_modificacion_1 es una copia independiente
df_modificacion_1 = df_filtrado[columnas_relevantes].copy()

# Definir las condiciones
condiciones = [
    df_modificacion_1['CARRERA CLASIFICACIÓN NIVEL 2'].isin(['Carreras Profesionales', 'Carreras Técnicas']),
    df_modificacion_1['CARRERA CLASIFICACIÓN NIVEL 2'] == 'Magister',
    df_modificacion_1['CARRERA CLASIFICACIÓN NIVEL 2'] == 'Doctorado'
]

# Definir las elecciones correspondientes a cada condición
elecciones = [
    'Pregrado',
    'Magister',
    'Doctorado'
]

# Crear la nueva columna 
df_modificacion_1['NIVEL'] = np.select(condiciones, elecciones)

# Duplicar cada fila del DataFrame, una para Hombres y otra para Mujeres
df_hombres = df_modificacion_1.copy()
df_mujeres = df_modificacion_1.copy()

# Asignar la categoría correspondiente a la nueva columna 'Sexo'
df_hombres['Sexo'] = 'Hombres'
df_mujeres['Sexo'] = 'Mujeres'

# Asignar a cada fila la cantidad de matriculados de cada sexo 
df_hombres['Matriculados'] = df_hombres['MATRICULADOS HOMBRES POR PROGRAMA']
df_mujeres['Matriculados'] = df_mujeres['MATRICULADOS MUJERES POR PROGRAMA']

# Eliminar las columnas de matriculados por sexo, ya que no son necesarias después de asignar la 'Cantidad'
df_hombres.drop(['CARRERA CLASIFICACIÓN NIVEL 2', 'MATRICULADOS MUJERES POR PROGRAMA', 'MATRICULADOS HOMBRES POR PROGRAMA'], axis=1, inplace=True)
df_mujeres.drop(['CARRERA CLASIFICACIÓN NIVEL 2', 'MATRICULADOS MUJERES POR PROGRAMA', 'MATRICULADOS HOMBRES POR PROGRAMA'], axis=1, inplace=True)

# Combinar los DataFrames de hombres y mujeres
df_modificacion_2 = pd.concat([df_hombres, df_mujeres], ignore_index=True)

# Eliminar filas donde la columna "Matriculados" tenga valores nulos
df_matricula_agregada = df_modificacion_2.dropna(subset=['Matriculados'])

# Mostrar el DataFrame resultante
print(df_matricula_agregada)

                                  CINE-F 2013 ÁREA     NIVEL     Sexo  \
0                              Artes y Humanidades  Pregrado  Hombres   
1      Ciencias Sociales, Periodismo e Información  Pregrado  Hombres   
2      Ciencias Sociales, Periodismo e Información  Pregrado  Hombres   
3      Ciencias Sociales, Periodismo e Información  Pregrado  Hombres   
5                                        Educación  Pregrado  Hombres   
...                                            ...       ...      ...   
24251         Administración de Empresas y Derecho  Pregrado  Mujeres   
24252         Administración de Empresas y Derecho  Pregrado  Mujeres   
24253         Administración de Empresas y Derecho  Pregrado  Mujeres   
24254         Administración de Empresas y Derecho  Pregrado  Mujeres   
24255         Administración de Empresas y Derecho  Pregrado  Mujeres   

       Matriculados  
0              22.0  
1              55.0  
2              16.0  
3              61.0  
5            

- ### **Grafica del indicador del 5 :** Porcentaje de personas matriculadas en educación superior, según sexo al total de personas matriculadas.

In [8]:
def calculo_distribucion(df):
    matriculados_mujeres = df[(df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matriculados_hombres = df[(df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    total_general        = df['Matriculados'].sum()

    return  matriculados_mujeres, matriculados_hombres, total_general

# Llamar a la función y almacenar 
matriculados_mujeres, matriculados_hombres, total_general = calculo_distribucion(df_matricula_agregada)

#calculo porcentual 
por_matriculados_mujeres        = (matriculados_mujeres / total_general) * 100
por_matriculados_hombres        = (matriculados_hombres / total_general) * 100


# Mostrar los resultados
print('Mujeres en Educación Superior:  {:.2f}%'.format(por_matriculados_mujeres), ' Cantidad:  {:.0f} matriculados'.format(matriculados_mujeres))
print('Hombres en Educación Superior:  {:.2f}%'.format(por_matriculados_hombres), ' Cantidad:  {:.0f} matriculados'.format(matriculados_hombres))

Mujeres en Educación Superior:  53.38%  Cantidad:  643870 matriculados
Hombres en Educación Superior:  46.62%  Cantidad:  562227 matriculados


- ### **Grafica del indicador 6:** Distribución de matrículas de pregrado, magíster y doctorado según sexo.

In [9]:
def calculo_distribucion(df):
    # Gráfico Distribución de matrículas de pregrado, magíster y doctorado según sexo
    matricula_Dr_mujeres        = df[(df['NIVEL'] == 'Doctorado') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_Dr_hombres        = df[(df['NIVEL'] == 'Doctorado') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_Dr_total          = df[(df['NIVEL'] == 'Doctorado')]['Matriculados'].sum()

    matricula_Mg_mujeres        = df[(df['NIVEL'] == 'Magister') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_Mg_hombres        = df[(df['NIVEL'] == 'Magister') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_Mg_total          = df[(df['NIVEL'] == 'Magister')]['Matriculados'].sum()

    matricula_pregrado_mujeres  = df[(df['NIVEL'] == 'Pregrado') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_pregrado_hombres  = df[(df['NIVEL'] == 'Pregrado') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_pregrado_total    = df[(df['NIVEL'] == 'Pregrado')]['Matriculados'].sum()

    return  matricula_Dr_mujeres, matricula_Dr_hombres, matricula_Dr_total, \
            matricula_Mg_mujeres, matricula_Mg_hombres, matricula_Mg_total, \
            matricula_pregrado_mujeres, matricula_pregrado_hombres, matricula_pregrado_total

# Llamar a la función y almacenar 
matricula_Dr_mujeres, matricula_Dr_hombres, matricula_Dr_total, matricula_Mg_mujeres, matricula_Mg_hombres, matricula_Mg_total, matricula_pregrado_mujeres, matricula_pregrado_hombres, matricula_pregrado_total = calculo_distribucion(df_matricula_agregada)

#calculo porcentual 
por_matricula_Dr_mujeres        = (matricula_Dr_mujeres / matricula_Dr_total) * 100
por_matricula_Dr_hombres        = (matricula_Dr_hombres / matricula_Dr_total) * 100

por_matricula_Mg_mujeres        = (matricula_Mg_mujeres / matricula_Mg_total) * 100
por_matricula_Mg_hombres        = (matricula_Mg_hombres / matricula_Mg_total) * 100

por_matricula_pregrado_mujeres  = (matricula_pregrado_mujeres / matricula_pregrado_total) * 100
por_matricula_pregrado_hombres  = (matricula_pregrado_hombres / matricula_pregrado_total) * 100

# Mostrar los resultados
print('Gráfico Distribución de matrículas de pregrado, magíster y doctorado según sexo.')
print('Matrículas de Mujeres en Doctorado: {:.2f}%'.format(por_matricula_Dr_mujeres), ' Cantidad:   {:.0f} personas'.format(matricula_Dr_mujeres))
print('Matrículas de Hombres en Doctorado: {:.2f}%'.format(por_matricula_Dr_hombres), ' Cantidad:   {:.0f} personas'.format(matricula_Dr_hombres))

print('Matrículas de Mujeres en Magister:  {:.2f}%'.format(por_matricula_Mg_mujeres), ' Cantidad:  {:.0f} personas'.format(matricula_Mg_mujeres))
print('Matrículas de Hombres en Magister:  {:.2f}%'.format(por_matricula_Mg_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_Mg_hombres))

print('Matrículas de Mujeres en Pregrado:  {:.2f}%'.format(por_matricula_pregrado_mujeres), ' Cantidad: {:.0f} personas'.format(matricula_pregrado_mujeres))
print('Matrículas de Hombres en Pregrado:  {:.2f}%'.format(por_matricula_pregrado_hombres), ' Cantidad: {:.0f} personas\n'.format(matricula_pregrado_hombres))

Gráfico Distribución de matrículas de pregrado, magíster y doctorado según sexo.
Matrículas de Mujeres en Doctorado: 42.68%  Cantidad:   2934 personas
Matrículas de Hombres en Doctorado: 57.32%  Cantidad:   3941 personas
Matrículas de Mujeres en Magister:  51.05%  Cantidad:  24239 personas
Matrículas de Hombres en Magister:  48.95%  Cantidad:  23240 personas
Matrículas de Mujeres en Pregrado:  53.54%  Cantidad: 616697 personas
Matrículas de Hombres en Pregrado:  46.46%  Cantidad: 535046 personas



- ### **Grafica del indicador 7:** Distribución de personas en pregrado según sexo por campo de educación.

In [10]:
def calculo_distribucion(df):
# Cantidad de matrículas anuales de magíster y doctorado en la universidades en Chile.
    matricula_AD_mujeres        = df[(df['CINE-F 2013 ÁREA'] == 'Administración de Empresas y Derecho') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_AD_hombres        = df[(df['CINE-F 2013 ÁREA'] == 'Administración de Empresas y Derecho') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_AD_total          = df[(df['CINE-F 2013 ÁREA'] == 'Administración de Empresas y Derecho')]['Matriculados'].sum()

    matricula_ASPV_mujeres      = df[(df['CINE-F 2013 ÁREA'] == 'Agricultura, Silvicultura, Pesca y Veterinaria') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_ASPV_hombres      = df[(df['CINE-F 2013 ÁREA'] == 'Agricultura, Silvicultura, Pesca y Veterinaria') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_ASPV_total        = df[(df['CINE-F 2013 ÁREA'] == 'Agricultura, Silvicultura, Pesca y Veterinaria')]['Matriculados'].sum()

    matricula_AH_mujeres        = df[(df['CINE-F 2013 ÁREA'] == 'Artes y Humanidades') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_AH_hombres        = df[(df['CINE-F 2013 ÁREA'] == 'Artes y Humanidades') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_AH_total          = df[(df['CINE-F 2013 ÁREA'] == 'Artes y Humanidades')]['Matriculados'].sum()

    matricula_CME_mujeres       = df[(df['CINE-F 2013 ÁREA'] == 'Ciencias naturales, matemáticas y estadística') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_CME_hombres       = df[(df['CINE-F 2013 ÁREA'] == 'Ciencias naturales, matemáticas y estadística') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_CME_total         = df[(df['CINE-F 2013 ÁREA'] == 'Ciencias naturales, matemáticas y estadística')]['Matriculados'].sum()

    matricula_CPI_mujeres       = df[(df['CINE-F 2013 ÁREA'] == 'Ciencias Sociales, Periodismo e Información') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_CPI_hombres       = df[(df['CINE-F 2013 ÁREA'] == 'Ciencias Sociales, Periodismo e Información') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_CPI_total         = df[(df['CINE-F 2013 ÁREA'] == 'Ciencias Sociales, Periodismo e Información')]['Matriculados'].sum()

    matricula_E_mujeres         = df[(df['CINE-F 2013 ÁREA'] == 'Educación') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_E_hombres         = df[(df['CINE-F 2013 ÁREA'] == 'Educación') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_E_total           = df[(df['CINE-F 2013 ÁREA'] == 'Educación')]['Matriculados'].sum()

    matricula_IIC_mujeres       = df[(df['CINE-F 2013 ÁREA'] == 'Ingeniería, Industria y Construcción') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_IIC_hombres       = df[(df['CINE-F 2013 ÁREA'] == 'Ingeniería, Industria y Construcción') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_IIC_total         = df[(df['CINE-F 2013 ÁREA'] == 'Ingeniería, Industria y Construcción')]['Matriculados'].sum()

    matricula_SB_mujeres        = df[(df['CINE-F 2013 ÁREA'] == 'Salud y Bienestar') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_SB_hombres        = df[(df['CINE-F 2013 ÁREA'] == 'Salud y Bienestar') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_SB_total          = df[(df['CINE-F 2013 ÁREA'] == 'Salud y Bienestar')]['Matriculados'].sum()

    matricula_S_mujeres         = df[(df['CINE-F 2013 ÁREA'] == 'Servicios') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_S_hombres         = df[(df['CINE-F 2013 ÁREA'] == 'Servicios') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_S_total           = df[(df['CINE-F 2013 ÁREA'] == 'Servicios')]['Matriculados'].sum()

    matricula_TIC_mujeres       = df[(df['CINE-F 2013 ÁREA'] == 'Tecnología de la Información y la Comunicación (TIC)') & (df['Sexo'] == 'Mujeres')]['Matriculados'].sum()
    matricula_TIC_hombres       = df[(df['CINE-F 2013 ÁREA'] == 'Tecnología de la Información y la Comunicación (TIC)') & (df['Sexo'] == 'Hombres')]['Matriculados'].sum()
    matricula_TIC_total         = df[(df['CINE-F 2013 ÁREA'] == 'Tecnología de la Información y la Comunicación (TIC)')]['Matriculados'].sum()

    return  matricula_AD_mujeres, matricula_AD_hombres, matricula_AD_total, \
            matricula_ASPV_mujeres, matricula_ASPV_hombres, matricula_ASPV_total, \
            matricula_AH_mujeres, matricula_AH_hombres, matricula_AH_total, \
            matricula_CME_mujeres, matricula_CME_hombres, matricula_CME_total, \
            matricula_CPI_mujeres, matricula_CPI_hombres, matricula_CPI_total, \
            matricula_E_mujeres, matricula_E_hombres, matricula_E_total, \
            matricula_IIC_mujeres, matricula_IIC_hombres, matricula_IIC_total, \
            matricula_SB_mujeres, matricula_SB_hombres, matricula_SB_total, \
            matricula_S_mujeres, matricula_S_hombres, matricula_S_total, \
            matricula_TIC_mujeres, matricula_TIC_hombres, matricula_TIC_total


# Llamar a la función y almacenar 
matricula_AD_mujeres, matricula_AD_hombres, matricula_AD_total, matricula_ASPV_mujeres, matricula_ASPV_hombres, matricula_ASPV_total, matricula_AH_mujeres, matricula_AH_hombres, matricula_AH_total, matricula_CME_mujeres, matricula_CME_hombres, matricula_CME_total, matricula_CPI_mujeres, matricula_CPI_hombres, matricula_CPI_total, matricula_E_mujeres, matricula_E_hombres, matricula_E_total, matricula_IIC_mujeres, matricula_IIC_hombres, matricula_IIC_total, matricula_SB_mujeres, matricula_SB_hombres, matricula_SB_total, matricula_S_mujeres, matricula_S_hombres, matricula_S_total, matricula_TIC_mujeres, matricula_TIC_hombres, matricula_TIC_total = calculo_distribucion(df_matricula_agregada)

#calculo porcentual 
por_matricula_AD_mujeres        = (matricula_AD_mujeres / matricula_AD_total) * 100
por_matricula_AD_hombres        = (matricula_AD_hombres / matricula_AD_total) * 100

por_matricula_ASPV_mujeres      = (matricula_ASPV_mujeres / matricula_ASPV_total) * 100
por_matricula_ASPV_hombres      = (matricula_ASPV_hombres / matricula_ASPV_total) * 100

por_matricula_AH_mujeres        = (matricula_AH_mujeres / matricula_AH_total) * 100
por_matricula_AH_hombres        = (matricula_AH_hombres / matricula_AH_total) * 100

por_matricula_CME_mujeres       = (matricula_CME_mujeres / matricula_CME_total) * 100
por_matricula_CME_hombres       = (matricula_CME_hombres / matricula_CME_total) * 100

por_matricula_CPI_mujeres       = (matricula_CPI_mujeres / matricula_CPI_total) * 100
por_matricula_CPI_hombres       = (matricula_CPI_hombres / matricula_CPI_total) * 100

por_matricula_E_mujeres         = (matricula_E_mujeres / matricula_E_total) * 100
por_matricula_E_hombres         = (matricula_E_hombres / matricula_E_total) * 100

por_matricula_IIC_mujeres       = (matricula_IIC_mujeres / matricula_IIC_total) * 100
por_matricula_IIC_hombres       = (matricula_IIC_hombres / matricula_IIC_total) * 100

por_matricula_SB_mujeres        = (matricula_SB_mujeres / matricula_SB_total) * 100
por_matricula_SB_hombres        = (matricula_SB_hombres / matricula_SB_total) * 100

por_matricula_S_mujeres         = (matricula_S_mujeres / matricula_S_total) * 100
por_matricula_S_hombres         = (matricula_S_hombres / matricula_S_total) * 100

por_matricula_TIC_mujeres       = (matricula_TIC_mujeres / matricula_TIC_total) * 100
por_matricula_TIC_hombres       = (matricula_TIC_hombres / matricula_TIC_total) * 100

total_general_mujeres = matricula_AD_mujeres+matricula_ASPV_mujeres+matricula_AH_mujeres+matricula_CME_mujeres+matricula_CPI_mujeres+matricula_E_mujeres+matricula_IIC_mujeres+matricula_SB_mujeres+matricula_S_mujeres+matricula_TIC_mujeres
total_general_hombres = matricula_AD_hombres+matricula_ASPV_hombres+matricula_AH_hombres+matricula_CME_hombres+matricula_CPI_hombres+matricula_E_hombres+matricula_IIC_hombres+matricula_SB_hombres+matricula_S_hombres+matricula_TIC_hombres
total_general = total_general_mujeres + total_general_hombres

por_total_general_mujeres = (total_general_mujeres / total_general) * 100
por_total_general_hombres = (total_general_hombres / total_general) * 100

# Mostrar los resultados

print('Cantidad de matrículas anuales de magíster y doctorado en la universidades en Chile.')

print('Matrículas de Mujeres en AD:   {:.2f}%'.format(por_matricula_AD_mujeres), ' Cantidad: {:.0f} personas'.format(matricula_AD_mujeres))
print('Matrículas de Hombres en AD:   {:.2f}%'.format(por_matricula_AD_hombres), ' Cantidad: {:.0f} personas'.format(matricula_AD_hombres))

print('Matrículas de Mujeres en ASPV: {:.2f}%'.format(por_matricula_ASPV_mujeres), ' Cantidad:  {:.0f} personas'.format(matricula_ASPV_mujeres))
print('Matrículas de Hombres en ASPV: {:.2f}%'.format(por_matricula_ASPV_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_ASPV_hombres))

print('Matrículas de Mujeres en AH:   {:.2f}%'.format(por_matricula_AH_mujeres), ' Cantidad:  {:.0f} personas'.format(matricula_AH_mujeres))
print('Matrículas de Hombres en AH:   {:.2f}%'.format(por_matricula_AH_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_AH_hombres))

print('Matrículas de Mujeres en CME:  {:.2f}%'.format(por_matricula_CME_mujeres), ' Cantidad:  {:.0f} personas'.format(matricula_CME_mujeres))
print('Matrículas de Hombres en CME:  {:.2f}%'.format(por_matricula_CME_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_CME_hombres))

print('Matrículas de Mujeres en CPI:  {:.2f}%'.format(por_matricula_CPI_mujeres), ' Cantidad:  {:.0f} personas'.format(matricula_CPI_mujeres))
print('Matrículas de Hombres en CPI:  {:.2f}%'.format(por_matricula_CPI_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_CPI_hombres))

print('Matrículas de Mujeres en E:    {:.2f}%'.format(por_matricula_E_mujeres), ' Cantidad:  {:.0f} personas'.format(matricula_E_mujeres))
print('Matrículas de Hombres en E:    {:.2f}%'.format(por_matricula_E_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_E_hombres))

print('Matrículas de Mujeres en IIC:  {:.2f}%'.format(por_matricula_IIC_mujeres), ' Cantidad:  {:.0f} personas'.format(matricula_IIC_mujeres))
print('Matrículas de Hombres en IIC:  {:.2f}%'.format(por_matricula_IIC_hombres), ' Cantidad: {:.0f} personas'.format(matricula_IIC_hombres))

print('Matrículas de Mujeres en SB:   {:.2f}%'.format(por_matricula_SB_mujeres), ' Cantidad: {:.0f} personas'.format(matricula_SB_mujeres))
print('Matrículas de Hombres en SB:   {:.2f}%'.format(por_matricula_SB_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_SB_hombres))

print('Matrículas de Mujeres en S:    {:.2f}%'.format(por_matricula_S_mujeres), ' Cantidad:  {:.0f} personas'.format(matricula_S_mujeres))
print('Matrículas de Hombres en S:    {:.2f}%'.format(por_matricula_S_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_S_hombres))

print('Matrículas de Mujeres en TIC:  {:.2f}%'.format(por_matricula_TIC_mujeres), ' Cantidad:   {:.0f} personas'.format(matricula_TIC_mujeres))
print('Matrículas de Hombres en TIC:  {:.2f}%'.format(por_matricula_TIC_hombres), ' Cantidad:  {:.0f} personas'.format(matricula_TIC_hombres))

print('Total general de Mujeres:      {:.2f}%'.format(por_total_general_mujeres), ' Cantidad: {:.0f} personas'.format(total_general_mujeres))
print('Total general de Hombres:      {:.2f}%'.format(por_total_general_hombres), ' Cantidad: {:.0f} personas'.format(total_general_hombres))


Cantidad de matrículas anuales de magíster y doctorado en la universidades en Chile.
Matrículas de Mujeres en AD:   55.02%  Cantidad: 149086 personas
Matrículas de Hombres en AD:   44.98%  Cantidad: 121869 personas
Matrículas de Mujeres en ASPV: 58.73%  Cantidad:  20900 personas
Matrículas de Hombres en ASPV: 41.27%  Cantidad:  14687 personas
Matrículas de Mujeres en AH:   54.89%  Cantidad:  28415 personas
Matrículas de Hombres en AH:   45.11%  Cantidad:  23348 personas
Matrículas de Mujeres en CME:  43.69%  Cantidad:  13325 personas
Matrículas de Hombres en CME:  56.31%  Cantidad:  17174 personas
Matrículas de Mujeres en CPI:  63.94%  Cantidad:  43163 personas
Matrículas de Hombres en CPI:  36.06%  Cantidad:  24345 personas
Matrículas de Mujeres en E:    79.20%  Cantidad:  84583 personas
Matrículas de Hombres en E:    20.80%  Cantidad:  22208 personas
Matrículas de Mujeres en IIC:  21.60%  Cantidad:  53463 personas
Matrículas de Hombres en IIC:  78.40%  Cantidad: 194099 personas
Matrí

In [11]:
# Obtener los primeros 5 resultados de df_matricula_agregada
primeros_cinco_resultados = df_matricula_agregada.head()

# Mostrar los primeros 5 resultados
print(primeros_cinco_resultados)

                              CINE-F 2013 ÁREA     NIVEL     Sexo  \
0                          Artes y Humanidades  Pregrado  Hombres   
1  Ciencias Sociales, Periodismo e Información  Pregrado  Hombres   
2  Ciencias Sociales, Periodismo e Información  Pregrado  Hombres   
3  Ciencias Sociales, Periodismo e Información  Pregrado  Hombres   
5                                    Educación  Pregrado  Hombres   

   Matriculados  
0          22.0  
1          55.0  
2          16.0  
3          61.0  
5           1.0  


<a id="subseccion-5-8"></a>

## 8. **[Distribución del personal académico de las instituciones de educación superior según sexo.](https://www.observa.minciencia.gob.cl/genero/formacion-y-capital-humano/porcentaje-de-mujeres-entre-el-personal-academico-de-las-instituciones-de-educacion-superior-respecto-al-total)**

 * **Archivo**: Personal_Academico_SIES_2022.xlsx
 * **Descripción**: En esta sección, se realiza la carga del archivo Excel y extrae las columnas A, C y D para convertirlo en un DataFrame. Mediante el uso de NumPy, se añade una columna adicional llamada "Tipo de institución". Esta nueva columna se categoriza en base a los valores de "Codigo IES". Para lograr una representación detallada de los datos de personal, el script duplica cada fila del DataFrame resultante para separar los datos de hombres y mujeres, asignando a cada uno la cantidad correspondiente de personal y ajustando la categorización de sexo. Las columnas originales de personal por género se eliminan después de este paso para evitar redundancias. El script finaliza combinando los DataFrames de hombres y mujeres en uno solo, mostrando los datos procesados por consola. Cabe destacar que la columna correspondiente al año se excluye en esta fase ya que en la carga de los datos se utiliza ese argumento.

[Volver al índice](#indice) 

In [12]:
# Abrir el archivo xlsx
personal_xlsx = openpyxl.load_workbook('Personal_Academico_SIES_2022.xlsx')

# Seleccionar la hoja 'BD_Académicos_JCE'
hoja = personal_xlsx['BD_Académicos_JCE']

# Inicializar una lista para almacenar los datos
datos = []

# Iterar sobre las filas de la hoja, empezando por la segunda para omitir el encabezado
for row in hoja.iter_rows(min_row=4):
    # Extraer los valores de las columnas A (1), C (3) y D (4)
    fila = [row[0].value, row[2].value, row[3].value]
    datos.append(fila)

# Cerrar el archivo xlsx
personal_xlsx.close()

# Convertir los datos a un DataFrame de pandas
df_modificacion_1 = pd.DataFrame(datos, columns=['Codigo IES', 'Total Mujeres', 'Total Hombres'])

# Eliminar las filas donde todas las columnas especificadas contienen NaN simultáneamente
df_modificacion_1.dropna(how='all', inplace=True)

# Definir las condiciones (Universidades pertenecientes al CRUCH por código IES)
condiciones = [
    df_modificacion_1['Codigo IES'].isin([3, 34, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 895, 896])
]

# Definir las elecciones correspondientes a cada condición
elecciones = [
    'CRUCH'
]

# Crear la nueva columna 'Tipo de institución'
df_modificacion_1['Tipo de institución'] = np.select(condiciones, elecciones, default= 'NO CRUCH')

# Duplicar cada fila del DataFrame, una para Hombres y otra para Mujeres
df_hombres = df_modificacion_1.copy()
df_mujeres = df_modificacion_1.copy()

# Asignar la categoría correspondiente a la nueva columna 'Sexo'
df_hombres['Sexo'] = 'Hombres'
df_mujeres['Sexo'] = 'Mujeres'

# Asignar a cada fila la cantidad de matriculados de cada sexo y eliminar las columnas innecesarias
df_hombres['Personal'] = df_hombres['Total Hombres']
df_mujeres['Personal'] = df_mujeres['Total Mujeres']

# Eliminar las columnas de matriculados por sexo, ya que no son necesarias después de asignar la 'Cantidad'
df_hombres.drop(['Codigo IES', 'Total Mujeres', 'Total Hombres'], axis=1, inplace=True)
df_mujeres.drop(['Codigo IES', 'Total Mujeres', 'Total Hombres'], axis=1, inplace=True)

# Combinar los DataFrames de hombres y mujeres
df_modificacion_2 = pd.concat([df_hombres, df_mujeres], ignore_index=True)

# Agrupar por 'Tipo de institución' y 'Sexo' y sumar la cantidad de personal
df_academicos = df_modificacion_2.groupby(['Tipo de institución', 'Sexo'])['Personal'].sum().reset_index()

# Mostrar el DataFrame para verificación
print(df_academicos)


  Tipo de institución     Sexo      Personal
0               CRUCH  Hombres  12195.413182
1               CRUCH  Mujeres   8137.480682
2            NO CRUCH  Hombres  14359.479545
3            NO CRUCH  Mujeres  13492.100455


<a id="seccion-6"></a>

# **III) Carga**

[Volver al índice](#indice) 

In [ ]:
# importamos las librerías respectivas y leemos la credencial de acceso
import psycopg2
import configparser

config = configparser.ConfigParser()
config.read('credencial_login.conf')
database = config['postgresql']['database']
user     = config['postgresql']['user']
password = config['postgresql']['password']
host     = config['postgresql']['host']
port     = config['postgresql']['port']

<a id="subseccion-6-2"></a>

## 2. **[Evolución de matrículas anuales en pregrado y postgrado en universidades en chile.](https://www.observa.minciencia.gob.cl/indicadores/formacion-y-capital-humano/evolucion-matricula-pregrado-magister-y-doctorado-en-universidades-en-chile)**

 * **Tabla BD:** SIES_matricula
 * modificar variable **primeros_cinco_resultados_matricula** a **df_matricula** 
 
[Volver al índice](#indice)  

In [ ]:
def verificarDato(cursor, año):
    query = "SELECT COUNT(*) FROM \"SIES_Matricula\" WHERE \"AÑO\" = %s"
    cursor.execute(query, (año,))
    resultado = cursor.fetchone()
    return resultado[0] > 0

def cargarDatos(connection, primeros_cinco_resultados_matricula):
   with connection.cursor() as cursor:
        if not verificarDato(cursor, '2022'):
            for _, row in primeros_cinco_resultados_matricula.iterrows():
                query = """
                INSERT INTO "SIES_Matricula" ("AÑO", "TOTAL MATRICULADOS", "MATRICULADOS MUJERES POR PROGRAMA", "MATRICULADOS HOMBRES POR PROGRAMA", "CLASIFICACIÓN INSTITUCIÓN NIVEL 1", "CLASIFICACIÓN INSTITUCIÓN NIVEL 2", "CLASIFICACIÓN INSTITUCIÓN NIVEL 3", "REGIÓN", "NOMBRE CARRERA", "ÁREA DEL CONOCIMIENTO", "CINE-F 1997 AREA", "CINE-F 1997 SUBAREA", "ÁREA CARRERA GENÉRICA", "CINE-F 2013 AREA", "CINE-F 2013 SUBAREA", "NIVEL GLOBAL", "CARRERA CLASIFICACIÓN NIVEL 1", "CARRERA CLASIFICACIÓN NIVEL 2", "AUX_NIVEL")
                VALUES ('2022', %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(query, tuple(row))
            print('Datos del indicador ingresado con éxito')
        else:
            print('Ya existe un registro para el año 2022')

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        database=database,
        user=user,
        password=password,
        host=host,
        port=port
    )
    connection.autocommit = True
    print('Conexión iniciada') 
    cargarDatos(connection, primeros_cinco_resultados_matricula)

except psycopg2.DatabaseError as e:
    print(f"Error al conectarse a la base de datos: {e}")
finally:
    if connection:
        connection.close()
        print('Conexión finalizada')

Error al conectarse a la base de datos: connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?



NameError: name 'connection' is not defined

<a id="subseccion-6-5"></a>

## 5. **[Porcentaje de personas matriculadas en educación superior, según sexo respecto al total de personas matriculadas.](https://www.observa.minciencia.gob.cl/genero/formacion-y-capital-humano/porcentaje-de-mujeres-matriculadas-en-la-educacion-superior-respecto-al-total)**

<a id="subseccion-6-6"></a>

## 6. **[Distribución de matrículas de pregrado, magíster y doctorado según sexo.](https://www.observa.minciencia.gob.cl/genero/formacion-y-capital-humano/porcentaje-de-mujeres-matriculadas-en-pregrado-magister-y-doctorado)**

<a id="subseccion-6-7"></a>

## 7. **[Distribución de personas matriculadas en pregrado según sexo por campo de educación.](https://www.observa.minciencia.gob.cl/genero/formacion-y-capital-humano/porcentaje-de-mujeres-matriculadas-en-pregrado-magister-y-doctorado-por-area-de-conocimiento)**

 * **Tabla BD:** SIES_Matricula_Agregada
 * modificar variable **primeros_cinco_resultados** a **df_matricula_agregada** 

[Volver al índice](#indice)   

In [ ]:
def verificarDato(cursor, año):
    query = "SELECT COUNT(*) FROM \"SIES_Matricula_Agregada\" WHERE \"Agno\" = %s"
    cursor.execute(query, (año,))
    resultado = cursor.fetchone()
    return resultado[0] > 0

def cargarDatos(connection, primeros_cinco_resultados):
   with connection.cursor() as cursor:
        if not verificarDato(cursor, '2022'):
            for _, row in primeros_cinco_resultados.iterrows():
                query = """
                INSERT INTO "SIES_Matricula_Agregada" ("Agno", "CINE F 2013", "Nivel", "Sexo", "Matriculados")
                VALUES ('2022', %s, %s, %s, %s)
                """
                cursor.execute(query, tuple(row))
            print('Datos del indicador ingresado con éxito')
        else:
            print('Ya existe un registro para el año 2022')

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        database=database,
        user=user,
        password=password,
        host=host,
        port=port
    )
    connection.autocommit = True
    print('Conexión iniciada') 
    cargarDatos(connection, primeros_cinco_resultados)

except psycopg2.DatabaseError as e:
    print(f"Error al conectarse a la base de datos: {e}")
finally:
    if connection:
        connection.close()
        print('Conexión finalizada')

Conexión iniciada
Datos del indicador ingresado con éxito
Conexión finalizada


<a id="subseccion-6-8"></a>

## 8. **[Distribución del personal académico de las instituciones de educación superior según sexo.](https://www.observa.minciencia.gob.cl/genero/formacion-y-capital-humano/porcentaje-de-mujeres-entre-el-personal-academico-de-las-instituciones-de-educacion-superior-respecto-al-total)**

 * **Tabla BD:** SIES_Academicos

[Volver al índice](#indice)    

In [ ]:
def verificarDato(cursor, año):
    query = "SELECT COUNT(*) FROM \"SIES_Academicos\" WHERE \"Año\" = %s"
    cursor.execute(query, (año,))
    resultado = cursor.fetchone()
    return resultado[0] > 0

def cargarDatos(connection, df_academicos):
   with connection.cursor() as cursor:
        if not verificarDato(cursor, '2022'):
            for _, row in df_academicos.iterrows():
                query = """
                INSERT INTO "SIES_Academicos" ("Año", "Tipo de institución", "Sexo", "Personal")
                VALUES ('2022', %s, %s, %s)
                """
                cursor.execute(query, tuple(row))
            print('Datos del indicador ingresado con éxito')
        else:
            print('Ya existe un registro para el año 2022')

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        database=database,
        user=user,
        password=password,
        host=host,
        port=port
    )
    connection.autocommit = True
    print('Conexión iniciada') 
    cargarDatos(connection, df_academicos)

except psycopg2.DatabaseError as e:
    print(f"Error al conectarse a la base de datos: {e}")
finally:
    if connection:
        connection.close()
        print('Conexión finalizada')

Conexión iniciada
Datos del indicador ingresado con éxito
Conexión finalizada
